## Deliverable 2. Create a Customer Travel Destinations Map.

In [63]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys, traceback
import numpy as np
import matplotlib as plt

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [64]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ushuaia,AR,-54.80,-68.30,39.20,93,75,31.09,moderate rain
1,1,Jamestown,US,42.10,-79.24,60.80,63,1,3.36,clear sky
2,2,Bengkulu,ID,-3.80,102.27,82.65,67,100,4.07,overcast clouds
3,3,Alice Springs,AU,-23.70,133.88,64.40,25,0,9.17,clear sky
4,4,Haines Junction,CA,60.75,-137.51,60.30,42,69,2.73,broken clouds


In [65]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [66]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Bengkulu,ID,-3.80,102.27,82.65,67,100,4.07,overcast clouds
6,6,Hilo,US,19.73,-155.09,82.40,65,90,10.29,overcast clouds
7,7,Portland,US,45.52,-122.68,86.00,35,75,3.36,broken clouds
13,13,Georgetown,MY,5.41,100.34,82.40,88,20,3.36,haze
16,16,Kapaa,US,22.08,-159.32,84.20,62,1,13.87,clear sky
17,17,Labuan,MY,5.28,115.24,80.60,88,20,1.12,few clouds
22,22,Tautira,PF,-17.73,-149.15,82.40,65,75,3.36,shower rain
24,24,Faanui,PF,-16.48,-151.75,78.35,77,7,20.13,light rain
25,25,Vaini,TO,-21.20,-175.20,75.20,88,90,10.29,overcast clouds
28,28,Banda Aceh,ID,5.56,95.32,83.05,71,94,4.29,overcast clouds


In [67]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [68]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [69]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,82.65,overcast clouds,-3.80,102.27,
6,Hilo,US,82.40,overcast clouds,19.73,-155.09,
7,Portland,US,86.00,broken clouds,45.52,-122.68,
13,Georgetown,MY,82.40,haze,5.41,100.34,
16,Kapaa,US,84.20,clear sky,22.08,-159.32,
17,Labuan,MY,80.60,few clouds,5.28,115.24,
22,Tautira,PF,82.40,shower rain,-17.73,-149.15,
24,Faanui,PF,78.35,light rain,-16.48,-151.75,
25,Vaini,TO,75.20,overcast clouds,-21.20,-175.20,
28,Banda Aceh,ID,83.05,overcast clouds,5.56,95.32,


In [70]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key,
    "location": "48.8566, 2.3522"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    #6d. Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6e. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # If an error is experienced, skip the city
    except:
        traceback.print_exc()
        pass

Traceback (most recent call last):
  File "<ipython-input-70-ec9483de83fd>", line 26, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-70-ec9483de83fd>", line 26, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-70-ec9483de83fd>", line 26, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-70-ec9483de83fd>", line 26, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-70-ec9483de83fd>", line 26, in <module>
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
IndexError: list inde

In [73]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,82.65,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
6,Hilo,US,82.40,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
7,Portland,US,86.00,broken clouds,45.52,-122.68,Portland Marriott City Center
13,Georgetown,MY,82.40,haze,5.41,100.34,Cititel Penang
16,Kapaa,US,84.20,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
17,Labuan,MY,80.60,few clouds,5.28,115.24,Tiara Labuan Hotel
22,Tautira,PF,82.40,shower rain,-17.73,-149.15,Pension Oaoa
24,Faanui,PF,78.35,light rain,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
25,Vaini,TO,75.20,overcast clouds,-21.20,-175.20,Keleti Beach Resort
28,Banda Aceh,ID,83.05,overcast clouds,5.56,95.32,OYO 884 Rumoh PMI Hotel


In [99]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'].str.strip() != '']
clean_hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df

C:\Users\tabit\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,82.65,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
6,Hilo,US,82.40,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
7,Portland,US,86.00,broken clouds,45.52,-122.68,Portland Marriott City Center
13,Georgetown,MY,82.40,haze,5.41,100.34,Cititel Penang
16,Kapaa,US,84.20,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
701,Miramar,US,80.60,clear sky,25.99,-80.23,Stadium Hotel
705,Roura,GF,77.00,clear sky,4.73,-52.33,Amazonie Découverte SARL
708,Yamada,JP,87.10,clear sky,39.47,141.95,Iroha-ya Inn
717,Wewak,PG,85.53,clear sky,-3.55,143.63,In Wewak Boutique Hotel


In [100]:
clean_hotel_df['Hotel Name'].value_counts()

Sheraton Kauai Resort at Coconut Beach    1
Novotel Dakar                             1
Mantra Trilogy Cairns                     1
Isles Sunset Lodge                        1
Bora-Bora Pearl Beach Resort & Spa        1
                                         ..
عطر و ادکلن پارسیس                        1
Taj Hotel                                 1
Leumerus Bungalows                        1
East Arnhem Real Estate                   1
محمد هادي ال فطيح                         1
Name: Hotel Name, Length: 216, dtype: int64

In [101]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [102]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather (Max Temp)</dt><dd>{Description} ({Max Temp} °F)</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))